In [ ]:
import pickle
from detectors import *
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import os

In [ ]:
def time_fit(det, data):
    time_begin = time.time()
        
    det = det.fit(np.array(data))
    
    return det, time.time() - time_begin

def time_test(det, data):
    time_begin = time.time()
        
    result = det.predict_proba(np.array(data))
    
    return det, time.time() - time_begin, result


# load data

In [ ]:
modes = ['bert_768', 'bow_50', 'bow_768']
subsets = {}
permutations = 10
max_length = 1000

result_pickle = 'data/results/twitter_diff_classes.pickle'

for mode in modes:
    subsets[mode] = {}
    with open('data/twitter/biden_{mode}_embeddings.pickle'.format(mode=mode), 'rb') as handle:
        data = pickle.load(handle)
        random.shuffle(data)
        data = [list(x) for x in zip(*data)][0][:max_length*permutations]
        data_permutated = [[x for x in data[i::permutations]] for i in range(permutations)]
        subsets[mode]['biden'] = data_permutated
    with open('data/twitter/trump_{mode}_embeddings.pickle'.format(mode=mode), 'rb') as handle:
        data = pickle.load(handle)
        random.shuffle(data)
        data = [list(x) for x in zip(*data)][0][:max_length*permutations]
        data_permutated = [[x for x in data[i::permutations]] for i in range(permutations)]
        subsets[mode]['trump'] = data_permutated
    

# initiate detectors

In [ ]:
detectors = {
    'csdd': CosineSimilarityDriftDetector(),
    'kts' : KernelTwoSampleDriftDetector(),
    'aks' : AlibiKSDetector(),
    'ammd': AlibiMMDDetector(),
    'lsdd': AlibiLSDDDetector(),
}

# tests

In [ ]:

if os.path.isfile(result_pickle):  # Do not overwrite
    print('Loading result pickle: ', result_pickle)
    with open(result_pickle, 'rb') as handle:
        results = pickle.load(handle)
else:
    results = {mode: {detector: {} for detector in detectors} for mode in modes}

    
for detector in detectors:
    for mode in modes:
        if not detector in results[mode]:
            results[mode][detector] = {}
        if 'predictions' in results[mode][detector]: # skip already computed
            continue
        
        results[mode][detector]['predictions'] = []
        results[mode][detector]['time_detect'] = []
        
        for perm in range(permutations):
            det, t = time_fit(detectors[detector], subsets[mode]['biden'][perm])
            results[mode][detector]['time_fit'] = t
            det_2, t, res = time_test(det, subsets[mode]['trump'][perm])
            results[mode][detector]['predictions'].append(res)
            results[mode][detector]['time_detect'].append(t)

with open(result_pickle, 'wb') as handle:
    pickle.dump(results, handle)

In [ ]:
print(results)